In [ ]:
"""
状態空間を Python でフルスクラッチで実装しようとしたもの。
いきなりフルスクラッチでやるのはやりすぎか（頑張りすぎ）かもしれないのでやめようか検討中。

"""
from dataclasses import dataclass
from typing import Callable

@dataclass
class StateSpace:
    """システムの状態空間表現を表すクラス"""

    nx: int
    """状態変数の次元"""

    nu : int
    """制御入力の次元"""

    ny : int
    """"出力の次元"""

    next : Callable
    """状態方程式の右辺の部分。

    時刻 `t` と状態 `x(t)` と制御入力 `u(t)` を受け取って
    dx/dt を表す関数
    """

    out : Callable
    """

    """

print("hello")

In [ ]:
"""
ほかのノートブックを読み込んで関数を使いまわす例
"""
from importnb import imports

with imports("ipynb"):
    from control import StateSpace

StateSpace(1, 2, 3)

In [ ]:
"""
穴あきバケツのシミュレーションを行う（行おうとしている）コード
ライブラリを使用せずにフルスクラッチで実装しようとしている
"""
from dataclasses import dataclass
from typing import Callable
from math import sqrt
from scipy.integrate import solve_ivp
from scipy.integrate._ivp.ivp import OdeResult

GRAV : float = 9.8
"""重力加速度 [m/s²]"""

@dataclass
class Bucket:
    """バケツを表すクラス。バケツ固有の情報を保持する。"""

    A: float
    """バケツの断面積 [m²]"""

@dataclass
class BucketSystem:
    """制御対象としてのバケツを表すクラス"""

    backet: Bucket
    """バケツの情報"""

    # water_level: Callable[[float], float]
    # """各時刻における水位"""

    flow_in: Callable[[float], float]
    """各時刻における水の流入量"""

    # def flow_out(self, t: float) -> float:
    #     """各時刻における水の流出量"""
    #     return sqrt(2 * GRAV * self.water_level(t))

    def ode(self, t:float, h:float) -> float:
        """水位の微分方程式。水位の導関数を表す。
        すなわち `h' = ode(t, h)` が成り立つ。
        """
        return (self.flow_in(t) - sqrt(2 * GRAV * h)) / self.backet.A

def run_simulation() -> OdeResult:
    """
    シミュレーションを実行する関数。
    バケツの断面積や水の流入量はここで与えている。
    """

    # バケツの情報
    backet = Bucket(A=5.0)

    # バケツの動的な挙動を表すオブジェクト
    system = BucketSystem(
        backet=backet,
        flow_in=lambda t: 10.0
    )

    # 水位の初期値
    h0 = 0.0

    # シミュレーションの時間範囲
    t_span = (0, 20)

    # シミュレーションの実行
    sol = solve_ivp(
        fun=lambda t, h: system.ode(t, h[0]),
        t_span=t_span,
        y0=[h0],
        max_step=0.01
    )

    return sol

run_simulation()